# 浪子之心科技
# 此代码仅为演示SadTalker驱动视频数字人


In [1]:
#@title 检测显卡
!nvidia-smi

Thu Jan  2 11:07:43 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#@title 一、下载源码并安装依赖
!git clone https://github.com/Zz-ww/SadTalker-Video-Lip-Sync

Cloning into 'SadTalker-Video-Lip-Sync'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 314 (delta 36), reused 32 (delta 32), pack-reused 260 (from 1)
Receiving objects: 100% (314/314), 4.56 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [3]:
# 安装miniconda,安装虚拟环境，激活虚拟环境。
# 如果已经下载，就可以不用再执行!wget
%cd /content/SadTalker-Video-Lip-Sync
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
# 设置包
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')
# 安装python=3.8 并检查python版本
!conda install python=3.8 -y
!python --version
# 创建虚拟环境
!conda create --name SadTalker-Video-Lip-Sync python=3.8 -y
# 激活环境
!source activate SadTalker-Video-Lip-Sync
# 安装依赖
%cd /content/SadTalker-Video-Lip-Sync
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install ffmpeg
!python3.8 -m pip install -r requirements.txt
!pip install ninja
# 安装DAIN模型
!python -m pip install paddlepaddle-gpu==2.3.2.post112  \
-f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

/content/SadTalker-Video-Lip-Sync
--2025-01-02 11:07:55--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   126MB/s    in 0.6s    

2025-01-02 11:07:56 (126 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
   

In [4]:
#@title 二、下载模型和数据
# 已经从云盘复制了文件夹过来就不需要运行
%cd /content/SadTalker-Video-Lip-Sync
!gdown https://drive.google.com/uc\?id\=1TB0QWxiGtagEbdwDIpIVeQftKtDBj8Q5
!rm -rf checkpoints
!unzip checkpoints.zip
!rm checkpoints.zip

/content/SadTalker-Video-Lip-Sync
Downloading...
From (original): https://drive.google.com/uc?id=1TB0QWxiGtagEbdwDIpIVeQftKtDBj8Q5
From (redirected): https://drive.google.com/uc?id=1TB0QWxiGtagEbdwDIpIVeQftKtDBj8Q5&confirm=t&uuid=45dc5ce2-96e8-44d9-895d-12d3929df2f3
To: /content/SadTalker-Video-Lip-Sync/checkpoints.zip
100% 4.38G/4.38G [00:50<00:00, 86.9MB/s]
Archive:  checkpoints.zip
   creating: checkpoints/
  inflating: checkpoints/auido2exp_00300-model.pth  
  inflating: checkpoints/auido2pose_00140-model.pth  
   creating: checkpoints/BFM_Fitting/
  inflating: checkpoints/BFM_Fitting/01_MorphableModel.mat  
  inflating: checkpoints/BFM_Fitting/BFM09_model_info.mat  
  inflating: checkpoints/BFM_Fitting/BFM_exp_idx.mat  
  inflating: checkpoints/BFM_Fitting/BFM_front_idx.mat  
  inflating: checkpoints/BFM_Fitting/Exp_Pca.bin  
  inflating: checkpoints/BFM_Fitting/facemodel_info.mat  
  inflating: checkpoints/BFM_Fitting/select_vertex_id.mat  
  inflating: checkpoints/BFM_Fitting/si

In [5]:
#@title 三、上传视频
%cd /content/SadTalker-Video-Lip-Sync/examples/driven_video
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content/SadTalker-Video-Lip-Sync/examples/driven_video


Saving yv22omynsa.mp4 to yv22omynsa.mp4


In [6]:
#@title 四、上传音频
%cd /content/SadTalker-Video-Lip-Sync/examples/driven_audio
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content/SadTalker-Video-Lip-Sync/examples/driven_audio


Saving luyin2.wav to luyin2.wav


In [7]:
#@title 五、进行推理
# 运行报错后，代码中将/content/SadTalker-Video-Lip-Sync/src/face3d/extract_kp_videos.py 第16行_2D 改为TWO_D
%cd /content/SadTalker-Video-Lip-Sync
!python inference.py --driven_audio '/content/SadTalker-Video-Lip-Sync/examples/driven_audio/luyin2.wav' \
            --source_video '/content/SadTalker-Video-Lip-Sync/examples/driven_video/1.mp4' \
            --enhancer 'lip' \
            --use_DAIN  \
              --time_step '0.5'

print('恭喜您高清数字人视频合并成功，结果在results中。')

/content/SadTalker-Video-Lip-Sync
Traceback (most recent call last):
  File "inference.py", line 8, in <module>
    from src.generate_batch import get_data
  File "/content/SadTalker-Video-Lip-Sync/src/generate_batch.py", line 7, in <module>
    import src.utils.audio as audio
  File "/content/SadTalker-Video-Lip-Sync/src/utils/audio.py", line 1, in <module>
    import librosa
  File "/usr/local/lib/python3.8/site-packages/librosa/__init__.py", line 209, in <module>
    from . import core
  File "/usr/local/lib/python3.8/site-packages/librosa/core/__init__.py", line 6, in <module>
    from .audio import *  # pylint: disable=wildcard-import
  File "/usr/local/lib/python3.8/site-packages/librosa/core/audio.py", line 8, in <module>
    import soundfile as sf
  File "/usr/local/lib/python3.8/site-packages/soundfile.py", line 17, in <module>
    from _soundfile import ffi as _ffi
  File "/usr/local/lib/python3.8/site-packages/_soundfile.py", line 2, in <module>
    import _cffi_backend
Impo